In [1]:
from typing import Dict, Text

import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

2023-04-02 20:47:46.908444: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-02 20:47:47.144347: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-02 20:47:47.199579: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-02 20:47:47.199596: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not ha

In [2]:
# Ratings data.
ratings = tfds.load('movielens/100k-ratings', split="train")
# Features of all the available movies.
movies = tfds.load('movielens/100k-movies', split="train")

# Select the basic features.
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"]
})
movies = movies.map(lambda x: x["movie_title"])

2023-04-02 21:38:34.438173: W tensorflow/core/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata".


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

...

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/1682 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/movielens/100k-movies/0.1.1.incomplete8PJEC4/movielens-train.tfrecord*...:…

Dataset movielens downloaded and prepared to /root/tensorflow_datasets/movielens/100k-movies/0.1.1. Subsequent calls will reuse this data.


In [27]:
for elem in ratings.take(2):
    print(elem)

{'movie_title': <tf.Tensor: shape=(), dtype=string, numpy=b"One Flew Over the Cuckoo's Nest (1975)">, 'user_id': <tf.Tensor: shape=(), dtype=string, numpy=b'138'>}
{'movie_title': <tf.Tensor: shape=(), dtype=string, numpy=b'Strictly Ballroom (1992)'>, 'user_id': <tf.Tensor: shape=(), dtype=string, numpy=b'92'>}


2023-04-02 21:52:41.801326: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [49]:
ratings_np = []
for rating in ratings:
    movie_title = rating['movie_title'].numpy()
    user_id = rating['user_id'].numpy()
    ratings_np.append({'movie_title': str(movie_title), 'user_id': int(user_id)})

In [51]:
import pandas as pd

ratings_df = pd.DataFrame(ratings_np)

In [26]:
for elem in movies.take(2):
    print(elem)

tf.Tensor(b'You So Crazy (1994)', shape=(), dtype=string)
tf.Tensor(b'Love Is All There Is (1996)', shape=(), dtype=string)


2023-04-02 21:52:35.868025: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [28]:
# Build vocabularies to convert user ids and movie titles into integer indices for embedding layers:
user_ids_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
user_ids_vocabulary.adapt(ratings.map(lambda x: x["user_id"]))

movie_titles_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
movie_titles_vocabulary.adapt(movies)

In [31]:
movie_titles_vocabulary.get_vocabulary()

['[UNK]',
 "Ulee's Gold (1997)",
 'That Darn Cat! (1997)',
 'Substance of Fire, The (1996)',
 'Sliding Doors (1998)',
 'Nightwatch (1997)',
 'Money Talks (1997)',
 'Kull the Conqueror (1997)',
 'Ice Storm, The (1997)',
 'Hurricane Streets (1998)',
 'Hugo Pool (1997)',
 'Fly Away Home (1996)',
 'Desperate Measures (1998)',
 'Designated Mourner, The (1997)',
 'Deceiver (1997)',
 'Chasing Amy (1997)',
 'Chairman of the Board (1998)',
 'Butcher Boy, The (1998)',
 'Body Snatchers (1993)',
 'Á köldum klaka (Cold Fever) (1994)',
 'unknown',
 'Zeus and Roxanne (1997)',
 "Young Poisoner's Handbook, The (1995)",
 'Young Guns II (1990)',
 'Young Guns (1988)',
 'Young Frankenstein (1974)',
 'You So Crazy (1994)',
 'Year of the Horse (1997)',
 'Yankee Zulu (1994)',
 'Wyatt Earp (1994)',
 'Wrong Trousers, The (1993)',
 'World of Apu, The (Apur Sansar) (1959)',
 "Wooden Man's Bride, The (Wu Kui) (1994)",
 'Wonderland (1997)',
 'Wonderful, Horrible Life of Leni Riefenstahl, The (1993)',
 'Women, The (

In [33]:
class MovieLensModel(tfrs.Model):
  # We derive from a custom base class to help reduce boilerplate. Under the hood,
  # these are still plain Keras Models.

    def __init__(
      self,
      user_model: tf.keras.Model,
      movie_model: tf.keras.Model,
      task: tfrs.tasks.Retrieval):
        super().__init__()

        # Set up user and movie representations.
        self.user_model = user_model
        self.movie_model = movie_model

        # Set up a retrieval task.
        self.task = task

    def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
        # Define how the loss is computed.

        user_embeddings = self.user_model(features["user_id"])
        movie_embeddings = self.movie_model(features["movie_title"])

        return self.task(user_embeddings, movie_embeddings) # query embeddings, candidate_embeddings

In [36]:
user_ids_vocabulary.vocabulary_size()

944

In [37]:
movie_titles_vocabulary.vocabulary_size()

1665

In [35]:
# Define user and movie models.
user_model = tf.keras.Sequential([
    user_ids_vocabulary,
    tf.keras.layers.Embedding(user_ids_vocabulary.vocabulary_size(), 64)
])
movie_model = tf.keras.Sequential([
    movie_titles_vocabulary,
    tf.keras.layers.Embedding(movie_titles_vocabulary.vocabulary_size(), 64)
])

# Define your objectives.
task = tfrs.tasks.Retrieval(metrics=tfrs.metrics.FactorizedTopK(
    movies.batch(128).map(movie_model)
  )
)

In [38]:
# Create a retrieval model.
model = MovieLensModel(user_model, movie_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))

In [39]:
# Train for 3 epochs.
model.fit(ratings.batch(4096), epochs=3)

Epoch 1/3
25/25 [==============================] - 6s 193ms/step - factorized_top_k/top_1_categorical_accuracy: 8.0000e-05 - factorized_top_k/top_5_categorical_accuracy: 0.0015 - factorized_top_k/top_10_categorical_accuracy: 0.0045 - factorized_top_k/top_50_categorical_accuracy: 0.0427 - factorized_top_k/top_100_categorical_accuracy: 0.0974 - loss: 33093.1991 - regularization_loss: 0.0000e+00 - total_loss: 33093.1991
Epoch 2/3
25/25 [==============================] - 5s 189ms/step - factorized_top_k/top_1_categorical_accuracy: 2.0000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0052 - factorized_top_k/top_10_categorical_accuracy: 0.0145 - factorized_top_k/top_50_categorical_accuracy: 0.1037 - factorized_top_k/top_100_categorical_accuracy: 0.2099 - loss: 31021.1987 - regularization_loss: 0.0000e+00 - total_loss: 31021.1987
Epoch 3/3
25/25 [==============================] - 5s 188ms/step - factorized_top_k/top_1_categorical_accuracy: 4.9000e-04 - factorized_top_k/top_5_categorica

In [40]:
# Use brute-force search to set up retrieval using the trained representations.
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index_from_dataset(
    movies.batch(100).map(lambda title: (title, model.movie_model(title))))

In [55]:
ratings_df[ratings_df['user_id'] == 100]

,movie_title,user_id
1368,"b'Rainmaker, The (1997)'",100
1756,b'Twisted (1996)',100
2596,b'Air Force One (1997)',100
5644,"b'Replacement Killers, The (1998)'",100
7170,b'Rosewood (1997)',100
9049,b'Sphere (1998)',100
10125,b'Desperate Measures (1998)',100
12478,b'Volcano (1997)',100
13375,b'Good Will Hunting (1997)',100
13959,b'Money Talks (1997)',100


In [57]:
# Get some recommendations.
scores, titles = index(np.array(["100"]))
print(f"Top 3 recommendations for user 42: {titles[0, :3]}")

Top 3 recommendations for user 42: [b'Senseless (1998)' b'N\xc3\xa9nette et Boni (1996)'
 b'Great Expectations (1998)']


In [58]:
scores

<tf.Tensor: shape=(1, 10), dtype=float32, numpy=
array([[6.2525997, 6.0429525, 5.715399 , 5.714102 , 5.6504297, 5.491537 ,
        5.291544 , 5.291544 , 5.27384  , 5.23478  ]], dtype=float32)>